# ChatGPT meets DALL-E on a dark, rainy night for humanity

<p>
Mal Minhas, v0.1, 03.12.22
</p>

> “How did you go bankrupt?” Bill asked.
> “Two ways,” Mike said. “Gradually and then suddenly.”

The quote is from Ernest Hemingway.  It relates to the impact of compounding.  Something similar is happening with AI capabilities right now.

### Installation

Install [`chatgpt-api`](https://github.com/taranjeet/chatgpt-api). It provides an unofficial local development setup that uses [`playwright`](https://playwright.dev/) automation and chromium to open a browser and parse HTML which allows us to interface to OpenAI's ChatGPT via REST.  Follow the installation instructions to configure it to run on port 5001 on localhost.

### The Sun Also Rises

Use `requests` to invoke a GET the `playwright` instance running on localhost via a function called `promptToText` which takes a quote and returns a response from ChatGPT.  We will use this to ask one of the oldest of questions, why is the sky blue?

In [1]:
import requests

def promptToText(prompt):
    params = {'q': prompt}
    headers = {'content-type' : 'application/json'}
    r = requests.get(f'http://localhost:5001/chat', params=params, headers=headers)
    assert(r.status_code == 200)
    return r.text

In [2]:
promptToText("why is the sky blue?")

"The sky appears blue to human observers because blue light from the sun is scattered in the Earth's atmosphere. This scattering is caused by the molecules and particles in the air, which are much smaller than the wavelengths of visible light. When light from the sun encounters these molecules and particles, it is scattered in all directions, with shorter wavelengths (such as blue and violet light) being scattered more effectively than longer wavelengths (such as red and orange light). This scattering causes the direct sunlight to appear white, but the sky itself takes on a blue hue because the blue light is scattered in all directions by the gases and particles in the Earth's atmosphere. The intensity of the blue color varies depending on the amount of dust, water vapor, and other substances in the atmosphere, which can cause the sky to appear more or less blue at different times. The blue color of the sky also changes throughout the day, appearing lighter at noon and darker in the mo

We can ask ChatGPT to construct an appropriate prompt for feeding into DALL-E and then extract that prompt as follows:

In [3]:
s = promptToText("Can you give me a prompt for DALL-E on how best to draw a view of a blue sky from my house")
s

'Sure, here is a possible prompt for DALL-E to generate an image of a blue sky as seen from your house:\n\nCopy code\nCreate an image of a beautiful blue sky as seen from a house. The house should be in the foreground of the image, with the sky stretching out behind it. Include some trees or other vegetation in the image to add depth and interest. The overall mood of the image should be peaceful and serene. Use a combination of light and dark shades of blue to give the sky a realistic appearance. Use a perspective that makes the sky appear vast and expansive, as if the viewer is looking up at it from the ground.\n\n\nThis prompt provides some specific details for DALL-E to use in generating the image, such as the inclusion of a house and trees in the image, the use of different shades of blue to create a realistic sky, and the use of a perspective that makes the sky appear large and expansive. It also conveys the desired mood of the image, which should be peaceful and serene. DALL-E ca

In [4]:
def extractPrompt(s):
    sblock = 'Copy code\n'
    start = s.find(sblock) + len(sblock)
    end = s[start:].find('\n\n')
    return s[start:start+end]

extractPrompt(s)

'Create an image of a beautiful blue sky as seen from a house. The house should be in the foreground of the image, with the sky stretching out behind it. Include some trees or other vegetation in the image to add depth and interest. The overall mood of the image should be peaceful and serene. Use a combination of light and dark shades of blue to give the sky a realistic appearance. Use a perspective that makes the sky appear vast and expansive, as if the viewer is looking up at it from the ground.'

Now we can feed this prompt into another method that invokes DALL-E.  Note for this to work, you need OpenAI credentials:

In [5]:
import os
import openai
import requests
import IPython.display

def configureEnvVars(key, org_id):
    # NB: host url is not prepended with \"https\" nor does it have a trailing slash.
    os.environ['OPENAI_API_KEY'] = key
    os.environ['OPENAI_ORG_ID'] = org_id

def getOrganisationId(file):
    with open(file) as f:
        key = f.read()
    return key
    
def getAPIKey(file):
    with open(file) as f:
        key = f.read()
    return key

def textToImage(text, target, show=False):
    response = openai.Image.create(
      prompt=text,
      n=1,
      size="1024x1024"
    )
    image_url = response['data'][0]['url']
    img_data = requests.get(image_url).content
    with open(target, 'wb') as handler:
        handler.write(img_data)
    return image_url

configureEnvVars(getAPIKey('.openAIKey'),getOrganisationId('.openAIOrg'))
openai.organization = os.getenv("OPENAI_ORG_ID")
openai.api_key = os.getenv("OPENAI_API_KEY")
models = openai.Model.list()

prompt_text = extractPrompt(s)
image_url = textToImage(prompt_text, 'chatgpt_image.png')
IPython.display.Image(url='chatgpt_image.png', width=400, height=300)